In [1]:
import os

In [2]:
%pwd

'D:\\CS\\iNeuron\\Deep Learning\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'D:\\CS\\iNeuron\\Deep Learning\\Chicken-Disease-Classification'

In [5]:
import tensorflow as tf
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [7]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.utils import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model='artifacts/training/model.h5',
            training_data='artifacts/data_ingestion/Chicken-fecal-images',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.score = None
        self.model = None
        self.config = config

    def _valid_generator(self):
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bi-linear'
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_generator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [10]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
   raise e

[2023-07-25 17:03:08,735: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-07-25 17:03:08,751: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-07-25 17:03:08,753: INFO: utils: created directory at: artifacts]
Found 1247 images belonging to 2 classes.
 2/20 [==>...........................] - ETA: 15:34 - loss: 0.0574 - accuracy: 0.9766

KeyboardInterrupt: 